In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [4]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor()    # X(feature vector) : float32
        self.y = torch.LongTensor()     # y(target) : int64
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}
    
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}, Target Shape: {self.y.shape}"

In [5]:
class TitanicTestDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor()    # X(feature vector) : float32
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}
    
    def __str__(self):
        return f"Data Size: {len(self.X)}, Input Shape: {self.X.shape}"

In [8]:
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path) # get train csv file from path
    test_df = pd.read_csv(test_data_path)   # get test csv file from path

    all_df = pd.concat([train_df, test_df], sort=False) # combine two datasets which shape are same

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Sruvived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)    # create dataset

    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])   # split data into train, validation
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

SyntaxError: invalid syntax (1035172348.py, line 25)